In [1]:
import pandas as pd
import glob

## Question 1: Open all .csv files in the folder.

In [2]:
folder_path = 'C:/Users/Falcons/Desktop/Python/Projects/KeithGalli/SalesAnalysis/Sales_Data'
csv_files = glob.glob(folder_path + '/*.csv')
df_list = (pd.read_csv(file) for file in csv_files)
df_all = pd.concat(df_list, ignore_index=True)

In [4]:
df_ordered = df_all.sort_values(by=['Order Date'], ascending=True)

In [5]:
df_ordered.describe()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
count,186305,186305,186305,186305,186305,186305
unique,178438,20,10,24,142396,140788
top,Order ID,USB-C Charging Cable,1,11.95,Order Date,Purchase Address
freq,355,21903,168552,21903,355,355


In [6]:
df_numbers = df_ordered.dropna(axis=0, how='any')

In [7]:
df_numbers.tail(356)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
32511,297674,AAA Batteries (4-pack),1,2.99,12/31/19 23:53,"425 Lake St, Portland, OR 97035"
51542,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
179487,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
9329,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
80956,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
...,...,...,...,...,...,...
26140,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
94767,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
186563,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
91799,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


In [8]:
df_2 = df_numbers.iloc[:-355]

In [9]:
df_2 = df_2.rename(columns={'Order Date': 'Date'})

In [10]:
df = df_2

In [11]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df.dtypes

Order ID                    object
Product                     object
Quantity Ordered            object
Price Each                  object
Date                datetime64[ns]
Purchase Address            object
dtype: object

## Question 2: What are maximum sales by month?

In [12]:
df['Month'] = df['Date'].dt.month
df['Price Each'] = df['Price Each'].astype('float64')
df['Quantity Ordered'] = df['Quantity Ordered'].astype('float64')
df['Sales'] = df['Price Each'] * df['Quantity Ordered']
monthly_sales = df.groupby('Month')['Sales'].sum()
max_month = monthly_sales.idxmax()
max_sales = monthly_sales.max()
print(f"Month with maximum sales: {max_month}")
print(f"Total sales in that month: {max_sales}")

Month with maximum sales: 12
Total sales in that month: 4613443.34


## Question 3: What are the maximum sales by city?

In [14]:
df['city'] = df['Purchase Address'].str.split(',', expand=True)[1] + df['Purchase Address'].str.split(',', expand=True)[2].str[:3]
df['city'] = df['city'].str.strip()
df['city'].unique()

array(['New York City NY', 'San Francisco CA', 'Dallas TX', 'Atlanta GA',
       'Boston MA', 'Seattle WA', 'Los Angeles CA', 'Portland OR',
       'Austin TX', 'Portland ME'], dtype=object)

In [15]:
monthly_sales = df.groupby('city')['Sales'].sum()
max_city = monthly_sales.idxmax()
max_sales = monthly_sales.max()
print(f"City with maximum sales: {max_city}")
print(f"Total sales in that month: {max_sales}")

City with maximum sales: San Francisco CA
Total sales in that month: 8262203.91


## Question 4: What products are often sold together?

In [35]:
df2 = df[df['Order ID'].duplicated(keep=False)]

In [40]:
df2['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))
df2 = df2[['Order ID','Grouped']].drop_duplicates()
df2.head(100)

,Order ID,Grouped
71311,144804,"Wired Headphones,iPhone"
74737,148074,"USB-C Charging Cable,Google Phone"
75124,148450,"Lightning Charging Cable,iPhone"
71176,144679,"Google Phone,USB-C Charging Cable"
74084,147451,"Google Phone,Wired Headphones"
...,...,...
73578,146975,"Bose SoundSport Headphones,Wired Headphones"
71596,145071,"AA Batteries (4-pack),USB-C Charging Cable"
72853,146277,"34in Ultrawide Monitor,Flatscreen TV"
69072,142683,"Wired Headphones,USB-C Charging Cable"


In [49]:
counts = df2.groupby('Grouped').size().sort_values(ascending=True)
print(counts)

Grouped
iPhone,iPhone,Lightning Charging Cable                      1
LG Washing Machine,Bose SoundSport Headphones               1
LG Washing Machine,Google Phone                             1
AAA Batteries (4-pack),Apple Airpods Headphones,iPhone      1
LG Washing Machine,Lightning Charging Cable                 1
                                                         ... 
iPhone,Wired Headphones                                   203
Google Phone,USB-C Charging Cable                         432
USB-C Charging Cable,Google Phone                         436
iPhone,Lightning Charging Cable                           442
Lightning Charging Cable,iPhone                           449
Length: 418, dtype: int64


In [45]:
counts.to_csv('repeated_entries_count.csv', index=True)

## Question 5: What product sold the most? Why do you think it sold the most?

In [53]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Date,Purchase Address,Month,Sales,city
73891,147268,Wired Headphones,1.0,11.99,2019-01-01 03:07:00,"9 Lake St, New York City, NY 10001",1,11.99,New York City NY
74701,148041,USB-C Charging Cable,1.0,11.95,2019-01-01 03:40:00,"760 Church St, San Francisco, CA 94016",1,11.95,San Francisco CA
76054,149343,Apple Airpods Headphones,1.0,150.00,2019-01-01 04:56:00,"735 5th St, New York City, NY 10001",1,150.00,New York City NY
76708,149964,AAA Batteries (4-pack),1.0,2.99,2019-01-01 05:53:00,"75 Jackson St, Dallas, TX 75001",1,2.99,Dallas TX
76061,149350,USB-C Charging Cable,2.0,11.95,2019-01-01 06:03:00,"943 2nd St, Atlanta, GA 30301",1,23.90,Atlanta GA


In [54]:
counts2 = df.groupby('Product')['Quantity Ordered'].sum().sort_values(ascending=True)
print(counts2)

Product
LG Dryer                        646.0
LG Washing Machine              666.0
Vareebadd Phone                2068.0
20in Monitor                   4129.0
ThinkPad Laptop                4130.0
Macbook Pro Laptop             4728.0
Flatscreen TV                  4819.0
Google Phone                   5532.0
34in Ultrawide Monitor         6199.0
27in 4K Gaming Monitor         6244.0
iPhone                         6849.0
27in FHD Monitor               7550.0
Bose SoundSport Headphones    13457.0
Apple Airpods Headphones      15661.0
Wired Headphones              20557.0
Lightning Charging Cable      23217.0
USB-C Charging Cable          23975.0
AA Batteries (4-pack)         27635.0
AAA Batteries (4-pack)        31017.0
Name: Quantity Ordered, dtype: float64


In [55]:
counts2.describe()

count       19.000000
mean     11004.157895
std       9673.951745
min        646.000000
25%       4429.000000
50%       6244.000000
75%      18109.000000
max      31017.000000
Name: Quantity Ordered, dtype: float64